<a href="https://colab.research.google.com/github/berndtlindner/R_and_python/blob/master/icepack_cvrp_1_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[cvrp-1-basic.ipynb](https://github.com/Icepack-co/examples/blob/master/python/cvrp-1-basic.ipynb)

# Capacitated Vehicle Routing Problem (CVRP)

A CVRP in an academic sense is to find the least number of vehicles which are required to service all demands at minimum cost. The way in which we emulate this is to add a large cost for using a vehicle and a smaller cost for travelling between nodes. Each vehicle is limited to the maximum amount that it can service (referred to as the capacity) and each node has a defined `quantity` for it's demand.

## Requirements
* **Protobuf** (Google's Protocol Buffers library) (https://pypi.org/project/protobuf/) `pip install protobuf`
* **Requests** (HTTP Python library) `pip install requests`
* **Plotting** (ipyleaflet) [installation instructions here](https://github.com/jupyter-widgets/ipyleaflet)
* Icepack API token of a key that has the Travelling Salesman Problem Solver enabled
* See the TSP/TSPTW examples for more details. If you can run those, you can run this example :-) 

## Schema details
If you're looking for more details around the schema fields and complete explanations check out the documentation on docs.scrudu.io


# Install

In [13]:
! git clone https://github.com/Icepack-co/examples.git
% cd /content/examples/python
! pip install ipyleaflet

# Ipyleaflet is not working on colab yet, using folium in the meantime
# https://stackoverflow.com/questions/50357314/outputting-an-ipleaflet-widget-in-google-colaboratory
# https://github.com/googlecolab/colabtools/issues/60
# ! jupyter nbextension enable --py --sys-prefix ipyleaflet
from ipyleaflet import Map, Polyline, Circle, LayerGroup
import folium
import pandas

Cloning into 'examples'...
remote: Enumerating objects: 450, done.
remote: Counting objects: 100% (450/450), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 450 (delta 267), reused 388 (delta 208), pack-reused 0
Receiving objects: 100% (450/450), 826.11 KiB | 4.98 MiB/s, done.
Resolving deltas: 100% (267/267), done.
/content/examples/python


# Pull Data

In [ ]:
df = pandas.read_csv('../sample_data/publist_orders.csv').head(10) # just grabbing the top 10 points.
df['quantity'] = 20       # create a quantity field of 20 units per node
df.loc[0,'quantity'] = 0  # set the quantity of the first node to zero (the depot)
print(df.head())

                    id         X          Y  pickupTime  dropoffTime  quantity
0  Guinness Storehouse -6.286811  53.341613           0            0         0
1  The Oval Bar Dublin -6.260296  53.348463           5           20        20
2   The Confession Box -6.258403  53.350342          10           15        20
3           Kehoes Pub -6.259429  53.341161          15           10        20
4      The Brazen Head -6.276332  53.344933          15           15        20


# Run

# Setup

In [12]:
# Enter the apiToken and we update the config.json (there are other better ways to do this)
%%writefile ../config.json
{"apiToken":"jGtuN3OuOYhnc2lGR3IrTHJdKLOCSz09KnhDjzK1S0w5P1BnlPebSog-0mBWyZxx7CfcGKJx6kjbFLMaoj7ZmHqcwn3ngLtfeKoJjL9ojjI3","endpoint":"https://api.test.icepack.ai/"}

Overwriting ../config.json


# Inputs

In [8]:
#@title Variables
import ipywidgets as widgets
number_of_vehicles = widgets.IntSlider(value=2, min=1, max=4, description="# of Vehicles")
vehicle_capacity = widgets.IntSlider(value=100, min=1, max=4, description="Capacity")

display(number_of_vehicles, vehicle_capacity)

IntSlider(value=2, description='# of Vehicles', max=4, min=1)

IntSlider(value=4, description='Capacity', max=4, min=1)

# Output

In [11]:
#@title Solve
exec(open('apiHelper.py').read()) # import some helper classes which we've written for you.

api = apiHelper(modelType="cvrp-jkfdoctmp51n") # set the model type to the cvrp model

sr = cvrp_jkfdoctmp51n_pb2.SolveRequest()
sr.solveType = 0

m = sr.model

def makePoint(row):
    p = cvrp_jkfdoctmp51n_pb2.Geocode()
    p.id = row.id
    p.x = row.X
    p.y = row.Y
    p.quantity  = row.quantity
    return (p)

# set the depot in the model
m.depot.CopyFrom(makePoint(df.loc[0,]))

# Add the balance of the data
for index, row in df.iterrows():
    if index != 0:
        m.points.append(makePoint(row))
        
m.NumberOfVehicles = number_of_vehicles.value
m.VehicleCapacity = vehicle_capacity.value
m.distancetype = 1

#print(m) # that's what the whole model looks like.

reqId = api.Post(sr)
#print(reqId) # if you want to see what the guid looks like.

sol = api.Get(reqId)

#print(sol.objective) # This is the cost of the solution returned by the api.

r = pandas.DataFrame(columns=['vehicleId', 'stop', 'QtyDelivered'])
g = list() # for storing the geometries
idx = 0
for i, e in enumerate(sol.routes):
    for j, s in enumerate(e.sequence):
        r.loc[idx] = list([i, s, e.visitCapacities[j]])
        idx+=1
    gs = list()
    for _, f in enumerate(e.edges):
        for _, p in enumerate(f.geometry):
            gs.append([p.y, p.x])
    g.append(gs)

# we can then left-join on the original data.    
r = pandas.merge(r,df,left_on='stop',right_on='id', how = 'left').drop(['id','quantity'], axis=1)

print(r)

cvec = list(['blue', 'red', 'orange'])

# form some locations to add to the map (just green dots seems ok)
locs = list()
for index, p in r.iterrows():
    circle = Circle()
    circle.location = (p['Y'], p['X'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

center = [r['Y'][1],r['X'][1]] # just use the first point as the center (i.e. the depot)
# m = Map(scroll_wheel_zoom=True, center=center, zoom=12)

# for i, gs in enumerate(g):
#     m.add_layer(Polyline(locations=gs,color=cvec[i],fill=False))

# m.add_layer(LayerGroup(layers=(locs)))

# m = folium.Map(
#     #width='50%', 
#     #height='50%',
#     location=(p['Y'], p['X']),
#     zoom_start=12,
#     #tiles='Stamen Terrain'
# )

# for i, gs in enumerate(g):
#     folium.PolyLine(locations=gs,color=cvec[i],fill=False, tooltip=i).add_to(m)
    
# m

NameError: ignored

# Map

In [ ]:
m = folium.Map(
    #width='50%', 
    #height='50%',
    location=(p['Y'], p['X']),
    zoom_start=12,
    #tiles='Stamen Terrain'
)

feature_groups = [folium.FeatureGroup("Vehicle 1"), folium.FeatureGroup("Vehicle 2")]

for i, gs in enumerate(g):
    folium.vector_layers.PolyLine(locations=gs,color=cvec[i],fill=False, tooltip=i).add_to(feature_groups[i])
    feature_groups[i].add_to(m)

folium.LayerControl().add_to(m)
m